## previous_application

In [1]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
save_files = True

In [ ]:
# load prev file
df_prev = pd.read_csv('input/previous_application.csv')

df_prev.set_index('SK_ID_PREV',inplace=True)

df_prev.info()
display(df_prev.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1670214 entries, 2030495 to 2418762
Data columns (total 36 columns):
SK_ID_CURR                     1670214 non-null int64
NAME_CONTRACT_TYPE             1670214 non-null object
AMT_ANNUITY                    1297979 non-null float64
AMT_APPLICATION                1670214 non-null float64
AMT_CREDIT                     1670213 non-null float64
AMT_DOWN_PAYMENT               774370 non-null float64
AMT_GOODS_PRICE                1284699 non-null float64
WEEKDAY_APPR_PROCESS_START     1670214 non-null object
HOUR_APPR_PROCESS_START        1670214 non-null int64
FLAG_LAST_APPL_PER_CONTRACT    1670214 non-null object
NFLAG_LAST_APPL_IN_DAY         1670214 non-null int64
RATE_DOWN_PAYMENT              774370 non-null float64
RATE_INTEREST_PRIMARY          5951 non-null float64
RATE_INTEREST_PRIVILEGED       5951 non-null float64
NAME_CASH_LOAN_PURPOSE         1670214 non-null object
NAME_CONTRACT_STATUS           1670214 non-null object
DAYS

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_prev.replace({365243:np.nan, 'XNA':np.nan, 'XAP':np.nan},inplace=True)

df_prev.rename(columns={'NAME_CONTRACT_TYPE':'CONTRACT',
                'AMT_ANNUITY':'ANNUITY',
                'AMT_APPLICATION':'AMT_APP',
                'AMT_CREDIT':'CREDIT',
                'AMT_DOWN_PAYMENT':'DOWNPAY',
                'AMT_GOODS_PRICE':'AMT_GOODS',
                'WEEKDAY_APPR_PROCESS_START':'WEEKDAY',
                'HOUR_APPR_PROCESS_START':'HOUR',
                'FLAG_LAST_APPL_PER_CONTRACT':'LAST_APP',
                'NFLAG_LAST_APPL_IN_DAY':'LAST_DAY',
                'NFLAG_MICRO_CASH':'MICRO',
                'RATE_DOWN_PAYMENT':'DOWNRATE',
                'RATE_INTEREST_PRIMARY':'INTEREST_PRIM',
                'RATE_INTEREST_PRIVILEGED':'INTEREST_PRIV',
                'NAME_CASH_LOAN_PURPOSE':'PURPOSE',
                'NAME_CONTRACT_STATUS':'STATUS',
                'DAYS_DECISION':'DAYS_DECIS',
                'NAME_PAYMENT_TYPE':'PAY_TYPE',
                'CODE_REFECT_REASON':'WHY_REJECT',
                'NAME_TYPE_SUITE':'SUITE',
                'NAME_CLIENT_TYPE':'CLIENT',
                'NAME_GOODS_CATEGORY':'GOODS_CAT',
                'NAME_PORTFOLIO':'FOLIO',
                'NAME_PRODUCT_TYPE':'PROD_TYPE',
                'CHANNEL_TYPE':'CHANNEL',
                'SELLERPLACE_AREA':'AREA',
                'NAME_SELLER_INDUSTRY':'INDUSTRY',
                'CNT_PAYMENT':'CNT_PAY',
                'NAME_YIELD_GROUP':'YIELD',
                'PRODUCT_COMBINATION':'PROD_COMB',
                'DAYS_FIRST_DRAWING':'FIRST_DRAW',
                'DAYS_FIRST_DUE':'FIRST_DUE',
                'DAYS_LAST_DUE_1ST_VERSION':'LAST_DUE_1ST',
                'DAYS_LAST_DUE':'LAST_DUE',
                'DAYS_TERMINATION':'TERMINATE',
                'NFLAG_INSURED_ON_APPROVAL':'INSURED'},
               inplace=True)

df_prev.head()

In [ ]:
cols_categ = df_prev.columns[df_prev.dtypes=='object']

display(df_prev[cols_categ].nunique())

In [ ]:
df_prev['CONTRACT'].replace({'Consumer loans':'Consumer',
                             'Cash loans':'Cash',
                             'Revolving loans':'Revolving'},
                            inplace=True)

df_prev['WEEKDAY'].replace({'MONDAY':1,
                            'TUESDAY':2,
                            'WEDNESDAY':3,
                            'THURSDAY':4,
                            'FRIDAY':5,
                            'SATURDAY':6,
                            'SUNDAY':7},
                            inplace=True)

df_prev['LAST_APP'].replace({'Y':1,'N':0},inplace=True)

df_prev['PURPOSE'].replace({ 'Everyday expenses':'Everyday',
                             'Car repairs':'CarRep',
                             'Building a house or an annex':'HouseBld',
                             'Purchase of electronic equipment':'Electronics',
                             'Payments on other loans':'Loans',
                             'Urgent needs':'Urgent',
                             'Buying a used car':'CarUsed',
                             'Buying a new car':'CarNew',
                             'Buying a holiday home / land':'HouseHol',
                             'Buying a home':'HouseBuy',
                             'Buying a garage':'Garage',
                             'Business development':'Business',
                             'Wedding / gift / holiday':'Wedding',
                             'Gasification / water supply':'Utilities',
                             'Refusal to name the goal':'Refused',
                             'Money for a third person':'3rdParty'},
                            inplace=True)

df_prev['STATUS'].replace({'Unused offer':'Unused'},inplace=True)

df_prev['PAY_TYPE'].replace({'Cash through the bank':'Cash',
                             'Non-cash from your account':'NonCash',
                             'Cashless from the account of the employer':'Employer'},
                            inplace=True)

df_prev['SUITE'].replace({'Unaccompanied':'Unacc',
                          'Spouse, partner':'Spouse',
                          'Other_B':'OthB',
                          'Other_A':'OthA',
                          'Group of people':'Group'},
                         inplace=True)

df_prev['GOODS_CAT'].replace({'Consumer Electronics':'Electronics',
                             'Construction Materials':'Materials',
                             'Auto Accessories':'AutoAcc',
                             'Photo / Cinema Equipment':'Photo',
                             'Audio/Video':'Audio',
                             'Clothing and Accessories':'Clothing',
                             'Sport and Leisure':'Sport',
                             'Medical Supplies':'Medical',
                             'Direct Sales':'Sales',
                             'Office Appliances':'Office',
                             'Additional Service':'Service',
                             'House Construction':'House'},
                             inplace=True)

df_prev['PROD_TYPE'].replace({'x-sell':0, 'walk-in':1},inplace=True)

df_prev['CHANNEL'].replace({'Country-wide':'CntryWide',
                            'Contact center':'Contact',
                            'Credit and cash offices':'CredCash',
                            'Regional / Local':'Local',
                            'AP+ (Cash loan)':'AP',
                            'Channel of corporate sales':'Corporate',
                            'Car dealer':'Car'},
                           inplace=True)

df_prev['INDUSTRY'].replace({'Consumer electronics':'Electronics',
                             'Auto technology':'Auto',
                             'MLM partners':'MLM'},
                            inplace=True)

df_prev['PROD_COMB'].replace({'POS mobile with interest':'Mobile_wint',
                             'Cash X-Sell: low':'CashXS_low',
                             'Cash X-Sell: high':'CashXS_high',
                             'Cash X-Sell: middle':'CashXS_mid',
                             'Cash Street: high':'CashSt_high',
                             'POS household without interest':'House_noint',
                             'POS household with interest':'House_wint',
                             'POS other with interest':'Other_wint',
                             'Card X-Sell':'CardXS',
                             'POS mobile without interest':'Mobile_noint',
                             'Card Street':'CardSt',
                             'POS industry with interest':'Ind_wint',
                             'Cash Street: low':'CashSt_low',
                             'POS industry without interest':'Ind_noint',
                             'Cash Street: middle':'CashSt_mid',
                             'POS others without interest':'Other_noint'},
                             inplace=True)

In [ ]:
cols_categ = df_prev.columns[df_prev.dtypes=='object']
df_prev[cols_categ].head()

In [ ]:
exclude_cols = ['SK_ID_CURR','STATUS']

num_cols = [col for col in df_prev.columns if col not in cols_categ and col not in exclude_cols]
df_prev = pd.get_dummies(df_prev,columns=cols_categ.drop('STATUS'))
dum_cols = [col for col in df_prev.columns if col not in num_cols and col not in exclude_cols]

In [ ]:
dum_funcs = {col:'mean' for col in dum_cols}
num_funcs = {col:['min','max','mean','sum'] for col in num_cols}

agg_funcs = {'SK_ID_CURR':'count'}
agg_funcs.update(dum_funcs)
agg_funcs.update(num_funcs)

In [ ]:
prev_grouped = df_prev.groupby(['SK_ID_CURR','STATUS']).agg(agg_funcs)

prev_grouped = prev_grouped.unstack(fill_value=0)
prev_grouped.columns = ['PREV_'+'_'.join(x) for x in prev_grouped.columns.values]
prev_cols = prev_grouped.columns

del df_prev

prev_grouped.head()

In [ ]:
drop_cols = [col for col in prev_grouped.columns if prev_grouped[col].abs().sum()==0]
prev_grouped.drop(drop_cols,axis=1,inplace=True)
print('Dropped',len(drop_cols),'columns.')

In [ ]:
print('loading train...')
df_train = pd.read_csv('input/application_train.csv',index_col='SK_ID_CURR')

print('merging train...')
df_train = df_train.merge(prev_grouped,how='left',left_index=True,right_index=True)

if save_files:
    print('saving train...')
    df_train[prev_cols].to_csv('input/app_train_ALL_prev.csv')
    
    print('loading test...')
    df_test = pd.read_csv('input/application_test.csv',index_col='SK_ID_CURR')

    print('merging test...')
    df_test = df_test.merge(prev_grouped,how='left',left_index=True,right_index=True)
   
    print('saving test...')
    df_test[prev_cols].to_csv('input/app_test_ALL_prev.csv')
    
del prev_grouped

display(df_train.head())

print('Done!')

## model

In [ ]:
def plot_roc_curve(y_true, y_pred, color='darkorange',plot_random=True,label='ROC curve'):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    score = roc_auc_score(y_true, y_pred)
    
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label=label+' (area = %0.2f)' % score)
    
    if plot_random:
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
target_counts = df_train.TARGET.value_counts()
targ1_weight = target_counts[0]/target_counts[1]
weights = df_train.TARGET.replace({0:1, 1:targ1_weight}).values

dtypes = df_train.dtypes
cols_categ = dtypes[dtypes=='object'].index.tolist()
for col in cols_categ:
    df_train[col] = df_train[col].astype('category')
    
y_true = df_train.TARGET
X = df_train.drop('TARGET',axis=1)
train_data = lgb.Dataset(X, label=y_true, weight=weights)

    
# train
param = {'objective':'binary', 'metric':'auc'}
bst = lgb.train(param, train_data)

#plot roc curve
plt.figure()
plot_roc_curve(y_true, bst.predict(X))

In [ ]:
plt.figure(figsize=(10,30))

feat_imp = pd.DataFrame({'split':bst.feature_importance(importance_type='split'),
                         'gain':bst.feature_importance(importance_type='gain')},
                        index=bst.feature_name())

total_gain = feat_imp['gain'].sum()
frac_gain = feat_imp['gain']/total_gain
frac_gain = frac_gain.sort_values(ascending=False)

display(frac_gain.head(30))

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.arange(len(frac_gain))+1, frac_gain)
plt.xlabel('n_features')
plt.ylabel('frac gain')

plt.subplot(1,2,2)
plt.plot(np.arange(len(frac_gain))+1,frac_gain.cumsum())
plt.xlabel('n_features')
plt.ylabel('cumulative frac gain')

In [ ]:
# select top n columns from pos_prev by importance
n_cols = 60
select_cols = frac_gain[frac_gain.index.str.contains('PREV_')].index[:n_cols]

if save_files:
    df_train[select_cols].to_csv('input/app_train_SELECT_prev.csv')
    df_test[select_cols].to_csv('input/app_test_SELECT_prev.csv')
    
frac_gain[select_cols]